In [1]:
using CSV, DataFrames

In [2]:
primAuthor = "Stoleriu"
fileOwnPapers = "LS_savedrecs_mine.txt"
fileCitations = "LS_savedrecs.txt"

"LS_savedrecs.txt"

In [3]:
root = dirname(@__FILE__)
inputAIS = joinpath(root, "AI_factor_full_1997-2021.txt")
inputISI = joinpath(root, "IF_factor_full_1997-2021.txt")

"/home/lali/TITAN-ROG-sync/julia/ISI/IF_factor_full_1997-2021.txt"

In [4]:
AIS = CSV.read(inputAIS, DataFrame; header=true)
AIS[8157, [1, 2, 3, 14, 25, 26]]

,Journal,1997,1998,2009,2020,2021
,String31,Float64,Float64,Float64,Float64,Float64
8157,IEEE MAGN LETT,0.789,0.789,0.789,0.409,0.409


In [5]:
ISI = CSV.read(inputISI, DataFrame; header=true)
ISI[8157, [1, 2, 3, 14, 25, 26]]

,Journal,1997,1998,2009,2020,2021
,String31,Float64,Float64,Float64,Float64,Float64
8157,IEEE MAGN LETT,1.692,1.692,1.692,1.549,1.549


In [6]:
root = dirname(@__FILE__)
inputMINE = joinpath(root, fileOwnPapers)
inputCIT = joinpath(root, fileCitations)

"/home/lali/TITAN-ROG-sync/julia/ISI/LS_savedrecs.txt"

In [7]:
MINEraw = CSV.read(inputMINE, DataFrame; header=true)
eltype.(eachcol(MINEraw));

┌ Warning: thread = 1 warning: parsed expected 67 columns, but didn't reach end of line around data row: 1. Parsing extra columns and widening final columnset
└ @ CSV /home/lali/.julia/packages/CSV/nofYz/src/file.jl:635


In [8]:
MINE = MINEraw[:, [2, 9, 24, 30, 32, 42, 45, 46, 47, 52, 53, 54, 55]];
eltype.(eachcol(MINE));

In [9]:
MINE[1, :PY] = 2021   # ADV MAT no year!!!!
eltype.(eachcol(MINE));

In [10]:
#root = dirname("/mnt/d/Stoleriu/")
#outputMINE = joinpath(root, "MINE_test.csv")
#CSV.write(outputMINE, MINE)

Pregatim MINE cu noi coloane

In [11]:
rename!(MINE,:"J9" => :"Journal")
insertcols!(MINE, :NA => 0);
insertcols!(MINE, :NEFF => 0.0);
insertcols!(MINE, :PRIM => 0);
insertcols!(MINE, :AI => 0.0);
insertcols!(MINE, :AINEFF => 0.0);
insertcols!(MINE, :IF => 0.0);
insertcols!(MINE, :IFNA => 0.0);
insertcols!(MINE, :TIMESC => 0);

Gasim numarul de autori si prim autor

In [12]:
for (row_index, lucrare) in enumerate(eachrow(MINE))
    MINE[row_index, :NA] = (1+count(";", lucrare[:AU]))

    if MINE[row_index, :NA] <= 5
        MINE[row_index, :NEFF] = MINE[row_index, :NA]
    elseif MINE[row_index, :NA] <= 15
        MINE[row_index, :NEFF] = (MINE[row_index, :NA] + 5.0) / 2.0
    elseif MINE[row_index, :NA] <= 75
        MINE[row_index, :NEFF] = (MINE[row_index, :NA] + 15.0) / 3.0
    else
        MINE[row_index, :NEFF] = (MINE[row_index, :NA] + 45.0) / 4.0
    end

    if findfirst(primAuthor, MINE[row_index, :AU])[1] == 1
        MINE[row_index, :PRIM] = 1
    elseif occursin(primAuthor, lucrare[:RP])
        MINE[row_index, :PRIM] = 1
    end
    #println(row_index, MINE[row_index, [:NA]])
    #println(row_index, MINE[row_index, [:PRIM]])
end

In [13]:
#MINE[:, [6, 7, 8, 12, 14, 15, 16, 17]]
MINE.PY = Int64.(MINE.PY)
MINE[ismissing.(MINE[!, :PY]), [1, 6, 7, 8, 12, 14, 15, 16, 17]]

,AU,Journal,PY,VL,AR,NA,NEFF,PRIM,AI
,String,String31?,Int64,Int64?,String7?,Int64,Float64,Int64,Float64


In [14]:
#root = dirname("/mnt/d/Stoleriu/")
#outputMINE = joinpath(root, "MINE_test.csv")
#CSV.write(outputMINE, MINE)

In [15]:
AISsubsect = filter(r -> any(occursin.(Vector(MINE[1, [:Journal]]), r.Journal)), AIS)
#subsect2 = filter(r -> (["ADV MATER"][1] .== r.Journal[1]), AIS)
size(AISsubsect, 1)
#["ADV MATER"][1] .== subsect[1, [:Journal]][1]

10

In [16]:
for (row_index, lucrare) in enumerate(eachrow(MINE))
    #lucrare = MINE[1, :]
    if(ismissing(lucrare[:Journal]))
        continue
    end
    for AISrow in eachrow(AIS)
        if( lucrare[:Journal] == AISrow[:Journal] )
            anString = string(lucrare[:PY])
            #println(AISrow[anString])
            MINE[row_index, :AI] = AISrow[anString]
            MINE[row_index, :AINEFF] = MINE[row_index, :AI] / MINE[row_index, :NEFF]
        end
    end
    for ISIrow in eachrow(ISI)
        if( lucrare[:Journal] == ISIrow[:Journal] )
            anString = string(lucrare[:PY])
            #println(ISIrow[anString])
            MINE[row_index, :IF] = ISIrow[anString]
            MINE[row_index, :IFNA] = ISIrow[anString] / MINE[row_index, :NA]
        end
    end
end

In [17]:
root = dirname("/mnt/d/Stoleriu/")
outputMINE = joinpath(root, "MINE_test.csv")
CSV.write(outputMINE, MINE)

"/mnt/d/Stoleriu/MINE_test.csv"

In [18]:
I = sum(MINE[!, :AINEFF])

14.938599034299036

In [19]:
P = sum(MINE[!, :PRIM] .* MINE[!, :AI])

10.668

############################
Citations
############################

In [20]:
MINECITEraw = CSV.read(inputCIT, DataFrame; header=true)
eltype.(eachcol(MINECITEraw));
# autocitari
#for (row_indexCIT, lucrareCIT) in enumerate(eachrow(MINECITEraw))
#    if( occursin(primAuthor, lucrareCIT.AU) )
#        println(lucrareCIT.AU)
#    end
#end

┌ Warning: thread = 1 warning: parsed expected 67 columns, but didn't reach end of line around data row: 1. Parsing extra columns and widening final columnset
└ @ CSV /home/lali/.julia/packages/CSV/nofYz/src/file.jl:635


68-element Vector{Type}:
 String1
 String
 Union{Missing, String63}
 Union{Missing, String}
 Union{Missing, String31}
 String
 Union{Missing, String63}
 Missing
 String
 String
 ⋮
 String
 String7
 String31
 Union{Missing, Int64}
 Union{Missing, String63}
 Union{Missing, String1}
 Union{Missing, String1}
 Dates.Date
 Missing

In [21]:
MINECITEraw = filter(row -> !(occursin(primAuthor, row.AU)),  MINECITEraw);
for (row_indexCIT, lucrareCIT) in enumerate(eachrow(MINECITEraw))
    if( occursin(primAuthor, lucrareCIT.AU) )
        println(lucrareCIT.AU)
    end
end

In [22]:
MINECITE = MINECITEraw[:, [2, 9, 30, 42, 45, 46, 47, 52, 53, 54, 55]];
eltype.(eachcol(MINECITE));

In [23]:
CITES = fill!(Vector{Vector{Int}}(undef, size(MINE)[1]), [0]);

In [28]:
for (row_indexMINE, lucrareMINE) in enumerate(eachrow(MINE))
    if ( ismissing(lucrareMINE[:DI]) )
        println("Skip paper $row_indexMINE")
        continue
    end
    println("Analyzing paper ", lucrareMINE[:DI])
    for (row_indexCIT, lucrareCIT) in enumerate(eachrow(MINECITE))
        if ( occursin(lucrareMINE[:DI], lucrareCIT[:CR]) )
            println("Paper $row_indexMINE cited by $row_indexCIT")
        end
    end
end

Analyzing paper 10.1002/adma.202105586
Analyzing paper 

10.1063/5.0042788
Paper 2 cited by 5
Paper 2 cited by 11
Analyzing paper 10.1109/LMAG.2021.3110488
Analyzing paper 

10.1016/j.physb.2020.412038
Analyzing paper 10.1016/j.jmmm.2019.04.080
Paper 5 cited by 104
Paper 5 cited by 109
Paper 5 cited by 143
Skip paper 6


Analyzing paper 10.1016/j.sna.2018.08.040
Analyzing paper 10.1016/j.jmmm.2017.08.084
Paper 8 cited by 77
Analyzing paper 10.1103/PhysRevB.96.064115
Paper 9 cited by 5


Paper 9 cited by 94
Paper 9 cited by 189
Paper 9 cited by 192
Paper 9 cited by 206
Paper 9 cited by 223
Paper 9 cited by 229
Paper 9 cited by 266
Analyzing paper 10.1103/PhysRevB.95.224107
Paper 10 cited by 5
Paper 10 cited by 10
Paper 10 cited by 15
Paper 10 cited by 22
Paper 10 cited by 35
Paper 10 cited by 50
Paper 10 cited by 96
Paper 10 cited by 114
Paper 10 cited by 129
Paper 10 cited by 135
Paper 10 cited by 138
Paper 10 cited by 146
Paper 10 cited by 168
Paper 10 cited by 190
Paper 10 cited by 192
Paper 10 cited by 266
Paper 10 cited by 276
Analyzing paper 10.1038/NMAT4606
Paper 11 cited by 5
Paper 11 cited by 14
Paper 11 cited by 15
Paper 11 cited by 18
Paper 11 cited by 23
Paper 11 cited by 24
Paper 11 cited by 38
Paper 11 cited by 42
Paper 11 cited by 43
Paper 11 cited by 44
Paper 11 cited by 45
Paper 11 cited by 57
Paper 11 cited by 63
Paper 11 cited by 66
Paper 11 cited by 69
Paper 11 cited by 70
Paper 11 cited by 71
Paper 11 cited by 75
Paper 11 cited by 76
Paper 11 cited

...

Paper 70 cited by 561
Analyzing paper 10.1063/1.1557656
Paper 71 cited by 78
Paper 71 cited by 109
Paper 71 cited by 125
Paper 71 cited by 142
Paper 71 cited by 182
Paper 71 cited by 185
Paper 71 cited by 333
Paper 71 cited by 354
Paper 71 cited by 360
Paper 71 cited by 378
Paper 71 cited by 460
Paper 71 cited by 480
Paper 71 cited by 482
Paper 71 cited by 490
Paper 71 cited by 496
Paper 71 cited by 500
Paper 71 cited by 515
Paper 71 cited by 533
Paper 71 cited by 540
Paper 71 cited by 541
Paper 71 cited by 548
Paper 71 cited by 553
Paper 71 cited by 566
Paper 71 cited by 570
Skip paper 72
Analyzing paper 10.1109/TMAG.2002.802823


Skip paper 74
Skip paper 75
Analyzing paper 10.1063/1.1447488
Analyzing paper 10.1016/S0921-4526(01)00984-X
Analyzing paper 10.1063/1.1355343
Paper 78 cited by 297


Paper 78 cited by 333
Paper 78 cited by 526
Paper 78 cited by 536
Paper 78 cited by 563
Paper 78 cited by 570
Analyzing paper 10.1016/S0304-8853(00)01353-6
Paper 79 cited by 560
Paper 79 cited by 566
Paper 79 cited by 570
Skip paper 80
Skip paper 81
Skip paper 82
Skip paper 83
Analyzing paper 10.1109/20.908799
Paper 84 cited by 357


Paper 84 cited by 408
Paper 84 cited by 553
Analyzing paper 10.1063/1.372768
Paper 85 cited by 408


Paper 85 cited by 553
Paper 85 cited by 560
Paper 85 cited by 568
Paper 85 cited by 572
Paper 85 cited by 573
Paper 85 cited by 574
Paper 85 cited by 575
Paper 85 cited by 576
Paper 85 cited by 577
Paper 85 cited by 578
Paper 85 cited by 579
Paper 85 cited by 580
Paper 85 cited by 581
Paper 85 cited by 582
Skip paper 86
Skip paper 87
Skip paper 88
